<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [1]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [ ]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
# os.environ["HOPSWORKS_API_KEY"] = ""
# with open('./data/hopsworks-api-key.txt', 'r') as file:
#     os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157269
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "netherlands", "city": "amsterdam", "street": "vondelpark", "aqicn_url": "https://api.waqi.info/feed/@2670", "latitude": 52.37, "longitude": 4.89}'

<!-- ### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span> -->

In [3]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [4]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,21.0,netherlands,amsterdam,vondelpark,2024-11-19,https://api.waqi.info/feed/@2670


In [7]:
air_quality_df = air_quality_fg.read()
air_quality_df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.17s) 


,date,pm25,pm25_lag_1,pm25_lag_2,pm25_lag_3,country,city,street,url
0,2018-05-04 00:00:00+00:00,19.0,22.0,34.0,16.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
1,2022-09-12 00:00:00+00:00,47.0,46.0,30.0,22.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
2,2014-12-14 00:00:00+00:00,49.0,23.0,24.0,23.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
3,2019-01-26 00:00:00+00:00,18.0,93.0,115.0,74.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
4,2021-12-17 00:00:00+00:00,27.0,44.0,38.0,45.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
...,...,...,...,...,...,...,...,...,...
3193,2018-09-21 00:00:00+00:00,16.0,28.0,26.0,39.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
3194,2015-12-12 00:00:00+00:00,27.0,32.0,44.0,30.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
3195,2024-11-18 00:00:00+00:00,15.0,18.0,52.0,33.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670
3196,2024-11-17 00:00:00+00:00,18.0,52.0,33.0,22.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670


In [8]:
yesterday = pd.Timestamp(today - datetime.timedelta(days=1)).tz_localize('UTC')
aq_yesterday_df = air_quality_df[air_quality_df['date'] == yesterday]
aq_yesterday_df

,date,pm25,pm25_lag_1,pm25_lag_2,pm25_lag_3,country,city,street,url
3195,2024-11-18 00:00:00+00:00,15.0,18.0,52.0,33.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670


In [ ]:
try:
    aq_today_df['pm25_lag_1'] = aq_yesterday_df['pm25'].iloc[0]
    aq_today_df['pm25_lag_2'] = aq_yesterday_df['pm25_lag_1'].iloc[0]
    aq_today_df['pm25_lag_3'] = aq_yesterday_df['pm25_lag_2'].iloc[0]
except Exception:
    # If there is no data for yesterday, fill with the mean value
    mean_value = air_quality_df['pm25'].mean().iloc[0]
    aq_today_df['pm25_lag_1'] = mean_value
    aq_today_df['pm25_lag_2'] = mean_value
    aq_today_df['pm25_lag_3'] = mean_value

In [10]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   pm25        1 non-null      float32       
 1   country     1 non-null      object        
 2   city        1 non-null      object        
 3   street      1 non-null      object        
 4   date        1 non-null      datetime64[ns]
 5   url         1 non-null      object        
 6   pm25_lag_1  1 non-null      float32       
 7   pm25_lag_2  1 non-null      float32       
 8   pm25_lag_3  1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 184.0+ bytes


In [11]:
aq_today_df = aq_today_df[air_quality_df.columns]
aq_today_df

,date,pm25,pm25_lag_1,pm25_lag_2,pm25_lag_3,country,city,street,url
0,2024-11-19,21.0,15.0,18.0,52.0,netherlands,amsterdam,vondelpark,https://api.waqi.info/feed/@2670


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [12]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 52.25°N 5.0°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-19,3.45,0.2,14.654254,27.824009,amsterdam
1,2024-11-20,3.55,0.0,19.211996,282.994629,amsterdam
2,2024-11-21,3.95,0.0,14.773164,223.025070,amsterdam
3,2024-11-22,2.80,0.7,17.782688,248.629303,amsterdam
4,2024-11-23,3.00,0.0,15.192682,166.293015,amsterdam
5,2024-11-24,15.75,0.0,33.501282,208.217255,amsterdam
6,2024-11-25,11.70,0.1,30.385365,216.326920,amsterdam
7,2024-11-26,9.85,0.1,17.727943,257.092590,amsterdam
8,2024-11-27,4.30,0.0,5.991594,147.264786,amsterdam
9,2024-11-28,6.30,0.0,10.464797,139.185013,amsterdam


In [13]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [14]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2024-11-19 10:41:13,804 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157269/fs/1147972/fg/1353068


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157269/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x30c1d5c90>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 676919
         }
       },
       "result": {
         "observed_value": 21.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T09:41:13.000804Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsucc

In [15]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-19 10:41:24,527 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157269/fs/1147972/fg/1340777


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157269/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x309c64110>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 664600
         }
       },
       "result": {
         "observed_value": 5.991593837738037,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-19T09:41:24.000526Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_mi

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
